In [1]:
from IPython.display import clear_output

In [2]:
from itertools import cycle
import torch
from model import QNetwork
from utils import *

In [3]:
def show_board(board, player):
    X = 1 if player == "X" else -1

    visual = []
    for b in board:
        if b == 0:
            visual.append(" ")
        elif b == X:
            visual.append("X")
        else:
            visual.append("O")
    return np.reshape(visual, SHAPE)

In [4]:
BOARD_SIZE = 9
SHAPE = (3, 3)
ROW_SIZE = SHAPE[0]

In [5]:
board = np.array([0] * BOARD_SIZE).astype(np.float32)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [7]:
model = QNetwork(BOARD_SIZE, BOARD_SIZE)
model.load_state_dict(torch.load("X.pth"))
model.eval()

QNetwork(
  (fc1): Linear(in_features=9, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=9, bias=True)
)

In [8]:
print(show_board(board, "X"), f"\nAgent: X")
it = cycle(("X", "O"))
while True:
    clear_output(wait=True)
    player = next(it)
    board = board * -1
    if player == "O":
        action = int(input("Position (0-8):"))
    else:
        state = torch.from_numpy(board).float().unsqueeze(0).to(device)
        action = np.argmax(model(state).cpu().data.numpy())
    outcome, board, row = check_outcome(board, action, SHAPE, ROW_SIZE)
    print(show_board(board, player), f"\nAgent: X")
    if outcome:
        print(outcome, player, row)
        break

[['O' 'X' ' ']
 ['O' 'X' ' ']
 [' ' 'X' ' ']] 
Agent: X
win X ((0, 1), (1, 1), (2, 1))
